In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np

import tensorflow as tf

sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))

np.random.seed(219)
tf.set_random_seed(219)

In [2]:
# Load training and eval data from tf.keras
(train_data, train_labels), (test_data, test_labels) = \
    tf.keras.datasets.mnist.load_data()

train_data = train_data[:50]
train_labels = train_labels[:50]
train_data = train_data / 255.
train_labels = np.asarray(train_labels, dtype=np.int32)

test_data = test_data[:50]
test_labels = test_labels[:50]
test_data = test_data / 255.
test_labels = np.asarray(test_labels, dtype=np.int32)

In [3]:
batch_size = 16

# for train
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
#train_dataset = train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.shuffle(buffer_size=10000, seed=None, reshuffle_each_iteration=False)
train_dataset = train_dataset.repeat(count=2)
train_dataset = train_dataset.batch(batch_size=batch_size)
print(train_dataset)

# for test
test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_labels))
test_dataset = test_dataset.shuffle(buffer_size = 10000)
test_dataset = test_dataset.repeat(count=2)
test_dataset = test_dataset.batch(batch_size = batch_size)
print(test_dataset)

<BatchDataset shapes: ((?, 28, 28), (?,)), types: (tf.float64, tf.int32)>
<BatchDataset shapes: ((?, 28, 28), (?,)), types: (tf.float64, tf.int32)>


## 1. `from_string_handle`

```python
@staticmethod
from_string_handle(
    string_handle,
    output_types,
    output_shapes=None,
    output_classes=None
)
```

Creates a new, uninitialized Iterator based on the given handle.

### 1.1 `make_one_shot_iterator()`

Creates an Iterator for enumerating the elements of this dataset.
* Note: The returned iterator will be initialized automatically. A "one-shot" iterator does not currently support re-initialization.

In [4]:
train_iterator = train_dataset.make_one_shot_iterator()
test_iterator = test_dataset.make_one_shot_iterator()

handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(
    handle, train_iterator.output_types)

x, y = iterator.get_next()
x = tf.cast(x, dtype = tf.float32)
y = tf.cast(y, dtype = tf.int32)

In [5]:
sess = tf.Session(config=sess_config)

train_iterator_handle = sess.run(train_iterator.string_handle())
test_iterator_handle = sess.run(test_iterator.string_handle())

# Train
max_epochs = 2
step = 0
for epoch in range(max_epochs):
  #sess.run(iterator.initializer) 할 필요 없음

  try:
    while True:
      train_labels_ = sess.run(y, feed_dict={handle: train_iterator_handle})
      test_labels_ = sess.run(y, feed_dict={handle: test_iterator_handle})
 
      print("step: %d  labels:" % step)
      print(train_labels_)
      print(test_labels_)
        
      step += 1

  except tf.errors.OutOfRangeError:
    print("End of dataset")  # ==> "End of dataset"

step: 0  labels:
[7 9 2 9 9 6 1 1 8 6 6 8 6 9 3 2]
[1 9 5 7 7 3 0 4 9 6 3 6 4 1 0 4]
step: 1  labels:
[0 4 9 7 2 3 3 0 5 1 0 7 1 9 0 4]
[4 3 1 9 1 7 2 2 5 0 0 0 5 7 6 7]
step: 2  labels:
[1 5 3 9 1 6 7 5 3 4 4 1 8 2 5 8]
[5 4 2 2 1 2 1 9 1 9 3 4 7 1 1 4]
step: 3  labels:
[3 3 7 9 2 9 9 6 1 1 8 6 6 8 6 9]
[4 4 4 4 1 0 1 3 0 7 4 6 9 1 7 4]
step: 4  labels:
[3 2 0 4 9 7 2 3 3 0 5 1 0 7 1 9]
[1 7 9 2 6 9 7 4 9 2 1 5 2 5 3 9]
step: 5  labels:
[0 4 1 5 3 9 1 6 7 5 3 4 4 1 8 2]
[3 6 4 1 0 7 2 4 4 5 0 2 4 0 1 7]
step: 6  labels:
[5 8 3 3]
[1 3 5 1]
End of dataset
End of dataset


### 1.2 `make_initializable_iterator()`

```python
make_initializable_iterator(shared_name=None)
```

Creates an Iterator for enumerating the elements of this dataset.

사용법
```python
dataset = ...
iterator = dataset.make_initializable_iterator()
# ...
sess.run(iterator.initializer)
```

In [6]:
train_iterator = train_dataset.make_initializable_iterator()
test_iterator = test_dataset.make_initializable_iterator()

handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(
    handle, train_iterator.output_types, train_iterator.output_shapes)

x, y = iterator.get_next()
x = tf.cast(x, dtype = tf.float32)
y = tf.cast(y, dtype = tf.int32)

In [7]:
sess = tf.Session(config=sess_config)

train_iterator_handle = sess.run(train_iterator.string_handle())
test_iterator_handle = sess.run(test_iterator.string_handle())

train_initializer = iterator.make_initializer(train_dataset)
test_initializer = iterator.make_initializer(test_dataset)

# Train
max_epochs = 2
step = 0
for epoch in range(max_epochs):
  sess.run(train_iterator.initializer)
  sess.run(test_iterator.initializer)
  
  try:
    while True:
      train_labels_ = sess.run(y, feed_dict={handle: train_iterator_handle})
      test_labels_ = sess.run(y, feed_dict={handle: test_iterator_handle})
 
      print("step: %d  labels:" % step)
      print(train_labels_)
      print(test_labels_)
        
      step += 1

  except tf.errors.OutOfRangeError:
    print("End of dataset")  # ==> "End of dataset"

step: 0  labels:
[7 9 2 9 9 6 1 1 8 6 6 8 6 9 3 2]
[1 9 5 7 7 3 0 4 9 6 3 6 4 1 0 4]
step: 1  labels:
[0 4 9 7 2 3 3 0 5 1 0 7 1 9 0 4]
[4 3 1 9 1 7 2 2 5 0 0 0 5 7 6 7]
step: 2  labels:
[1 5 3 9 1 6 7 5 3 4 4 1 8 2 5 8]
[5 4 2 2 1 2 1 9 1 9 3 4 7 1 1 4]
step: 3  labels:
[3 3 7 9 2 9 9 6 1 1 8 6 6 8 6 9]
[4 4 4 4 1 0 1 3 0 7 4 6 9 1 7 4]
step: 4  labels:
[3 2 0 4 9 7 2 3 3 0 5 1 0 7 1 9]
[1 7 9 2 6 9 7 4 9 2 1 5 2 5 3 9]
step: 5  labels:
[0 4 1 5 3 9 1 6 7 5 3 4 4 1 8 2]
[3 6 4 1 0 7 2 4 4 5 0 2 4 0 1 7]
step: 6  labels:
[5 8 3 3]
[1 3 5 1]
End of dataset
step: 7  labels:
[7 9 2 9 9 6 1 1 8 6 6 8 6 9 3 2]
[1 9 5 7 7 3 0 4 9 6 3 6 4 1 0 4]
step: 8  labels:
[0 4 9 7 2 3 3 0 5 1 0 7 1 9 0 4]
[4 3 1 9 1 7 2 2 5 0 0 0 5 7 6 7]
step: 9  labels:
[1 5 3 9 1 6 7 5 3 4 4 1 8 2 5 8]
[5 4 2 2 1 2 1 9 1 9 3 4 7 1 1 4]
step: 10  labels:
[3 3 7 9 2 9 9 6 1 1 8 6 6 8 6 9]
[4 4 4 4 1 0 1 3 0 7 4 6 9 1 7 4]
step: 11  labels:
[3 2 0 4 9 7 2 3 3 0 5 1 0 7 1 9]
[1 7 9 2 6 9 7 4 9 2 1 5 2 5 3 9]
step: 12  l

## 2. `from_structure`

```python
@staticmethod
from_structure(
    output_types,
    output_shapes=None,
    shared_name=None,
    output_classes=None
)
```

Creates a new, uninitialized Iterator with the given structure.

### 2.1 `make_one_shot_iterator()`

Creates an Iterator for enumerating the elements of this dataset.
* Note: The returned iterator will be initialized automatically. A "one-shot" iterator does not currently support re-initialization.

In [8]:
train_iterator = train_dataset.make_one_shot_iterator()
test_iterator = test_dataset.make_one_shot_iterator()

iterator = tf.data.Iterator.from_structure(train_iterator.output_types)

train_initializer = iterator.make_initializer(train_dataset)
test_initializer = iterator.make_initializer(test_dataset)

x, y = iterator.get_next()

sess = tf.Session(config=sess_config)
# Train for `num_epochs`, where for each epoch, we first iterate over
# dataset_range, and then iterate over dataset_evens.
for _ in range(2):
  # Initialize the iterator to `dataset_range`
  print('train')
  sess.run(train_initializer)
  while True:
    try:
      y_ = sess.run(y)
      print(y_)
    except tf.errors.OutOfRangeError:
      break
      
  print('test')
  sess.run(test_initializer)
  while True:
    try:
      y_ = sess.run(y)
      print(y_)
    except tf.errors.OutOfRangeError:
      break

train
[7 9 2 9 9 6 1 1 8 6 6 8 6 9 3 2]
[0 4 9 7 2 3 3 0 5 1 0 7 1 9 0 4]
[1 5 3 9 1 6 7 5 3 4 4 1 8 2 5 8]
[3 3 7 9 2 9 9 6 1 1 8 6 6 8 6 9]
[3 2 0 4 9 7 2 3 3 0 5 1 0 7 1 9]
[0 4 1 5 3 9 1 6 7 5 3 4 4 1 8 2]
[5 8 3 3]
test
[1 9 5 7 7 3 0 4 9 6 3 6 4 1 0 4]
[4 3 1 9 1 7 2 2 5 0 0 0 5 7 6 7]
[5 4 2 2 1 2 1 9 1 9 3 4 7 1 1 4]
[4 4 4 4 1 0 1 3 0 7 4 6 9 1 7 4]
[1 7 9 2 6 9 7 4 9 2 1 5 2 5 3 9]
[3 6 4 1 0 7 2 4 4 5 0 2 4 0 1 7]
[1 3 5 1]
train
[7 9 2 9 9 6 1 1 8 6 6 8 6 9 3 2]
[0 4 9 7 2 3 3 0 5 1 0 7 1 9 0 4]
[1 5 3 9 1 6 7 5 3 4 4 1 8 2 5 8]
[3 3 7 9 2 9 9 6 1 1 8 6 6 8 6 9]
[3 2 0 4 9 7 2 3 3 0 5 1 0 7 1 9]
[0 4 1 5 3 9 1 6 7 5 3 4 4 1 8 2]
[5 8 3 3]
test
[1 9 5 7 7 3 0 4 9 6 3 6 4 1 0 4]
[4 3 1 9 1 7 2 2 5 0 0 0 5 7 6 7]
[5 4 2 2 1 2 1 9 1 9 3 4 7 1 1 4]
[4 4 4 4 1 0 1 3 0 7 4 6 9 1 7 4]
[1 7 9 2 6 9 7 4 9 2 1 5 2 5 3 9]
[3 6 4 1 0 7 2 4 4 5 0 2 4 0 1 7]
[1 3 5 1]


### 2.2 `make_initializable_iterator()`

```python
make_initializable_iterator(shared_name=None)
```

Creates an Iterator for enumerating the elements of this dataset.

사용법
```python
dataset = ...
iterator = dataset.make_initializable_iterator()
# ...
sess.run(iterator.initializer)
```

In [9]:
train_iterator = train_dataset.make_initializable_iterator()
test_iterator = test_dataset.make_initializable_iterator()

iterator = tf.data.Iterator.from_structure(train_iterator.output_types)

train_initializer = iterator.make_initializer(train_dataset)
test_initializer = iterator.make_initializer(test_dataset)

x, y = iterator.get_next()

sess = tf.Session(config=sess_config)
# Train for `num_epochs`, where for each epoch, we first iterate over
# dataset_range, and then iterate over dataset_evens.
for _ in range(2):
  # Initialize the iterator to `dataset_range`
  print('train')
  sess.run(train_initializer)
  while True:
    try:
      y_ = sess.run(y)
      print(y_)
    except tf.errors.OutOfRangeError:
      break
      
  print('test')
  sess.run(test_initializer)
  while True:
    try:
      y_ = sess.run(y)
      print(y_)
    except tf.errors.OutOfRangeError:
      break

train
[7 9 2 9 9 6 1 1 8 6 6 8 6 9 3 2]
[0 4 9 7 2 3 3 0 5 1 0 7 1 9 0 4]
[1 5 3 9 1 6 7 5 3 4 4 1 8 2 5 8]
[3 3 7 9 2 9 9 6 1 1 8 6 6 8 6 9]
[3 2 0 4 9 7 2 3 3 0 5 1 0 7 1 9]
[0 4 1 5 3 9 1 6 7 5 3 4 4 1 8 2]
[5 8 3 3]
test
[1 9 5 7 7 3 0 4 9 6 3 6 4 1 0 4]
[4 3 1 9 1 7 2 2 5 0 0 0 5 7 6 7]
[5 4 2 2 1 2 1 9 1 9 3 4 7 1 1 4]
[4 4 4 4 1 0 1 3 0 7 4 6 9 1 7 4]
[1 7 9 2 6 9 7 4 9 2 1 5 2 5 3 9]
[3 6 4 1 0 7 2 4 4 5 0 2 4 0 1 7]
[1 3 5 1]
train
[7 9 2 9 9 6 1 1 8 6 6 8 6 9 3 2]
[0 4 9 7 2 3 3 0 5 1 0 7 1 9 0 4]
[1 5 3 9 1 6 7 5 3 4 4 1 8 2 5 8]
[3 3 7 9 2 9 9 6 1 1 8 6 6 8 6 9]
[3 2 0 4 9 7 2 3 3 0 5 1 0 7 1 9]
[0 4 1 5 3 9 1 6 7 5 3 4 4 1 8 2]
[5 8 3 3]
test
[1 9 5 7 7 3 0 4 9 6 3 6 4 1 0 4]
[4 3 1 9 1 7 2 2 5 0 0 0 5 7 6 7]
[5 4 2 2 1 2 1 9 1 9 3 4 7 1 1 4]
[4 4 4 4 1 0 1 3 0 7 4 6 9 1 7 4]
[1 7 9 2 6 9 7 4 9 2 1 5 2 5 3 9]
[3 6 4 1 0 7 2 4 4 5 0 2 4 0 1 7]
[1 3 5 1]
